# Tratamento de Dados

**TAREFA**

Webscraping, coleta e preparação dos DataFrames.

Os dados consistem em uma base que contém as features:
* IGP-M,
* Var. % no PIB nominal, 
* Spread NTN-B, inclinação da curva 
* indicador de concessão de crédito.

 Como target, será importado o desempenho do **ETF FIND11**, que segue o IFNC, índice de desempenho das maiores instituições financeiras do país. 

### Importação de Bibliotecas

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf

### Importação Target

In [25]:
data = yf.download("FIND11.SA",
                   start = '2000-01-01',
                   period = "max", # 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
                   interval = "1mo") # 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
                   
target = data["Adj Close"].pct_change().dropna()
target = pd.DataFrame(target)

[*********************100%***********************]  1 of 1 completed


### Importação Features

Fontes: 

https://www3.bcb.gov.br/sgspub/localizarseries/localizarSeries.do?method=prepararTelaLocalizarSeries

https://dadosabertos.bcb.gov.br/dataset/20542-saldo-da-carteira-de-credito-com-recursos-livres---total/resource/6e2b0c97-afab-4790-b8aa-b9542923cf88

http://www.ipeadata.gov.br/ExibeSerie.aspx?serid=521274780&module=M

In [7]:
def get_BCB(codigo_serie):
    url = f"https://api.bcb.gov.br/dados/serie/bcdata.sgs.{codigo_serie}/dados?formato=json"
    df = pd.read_json(url)
    df["data"] = pd.to_datetime(df["data"], dayfirst = True)
    df.set_index("data", inplace = True)
    df["valor"] = df["valor"].astype(float)
    return df

def get_pib(url):
    pib = pd.read_html(url)[2]
    pib.columns = pib.loc[0]
    pib.drop(0, axis = 0, inplace = True)
    pib.reset_index(drop = True, inplace = True)

    datas, pibs = [], []
    for i in range(len(pib)):
        data_iter = pib.iloc[i, 0].replace(".", "")
        pib_iter = pib.iloc[i, 1].replace(".", "")              
        pib_iter = float(pib_iter.replace(",", ""))/10000    
        datas.append(data_iter)
        pibs.append(pib_iter)

    pib["Data"] = pd.to_datetime(datas, format = "%Y%m")
    pib["PIB"] = pibs
    pib.set_index("Data", inplace = True)
    
    return pib

In [26]:
igpm.columns = ["IGP"]
credito_pib.columns = ["CRED/PIB"]
div_liq.columns = ["DIV_LIQ"]
target.columns = ["FIND11"]

In [4]:
igpm = get_BCB(189)
credito_pib = get_BCB(20622)
div_liq = get_BCB(4536)
pib = get_pib("http://www.ipeadata.gov.br/ExibeSerie.aspx?serid=521274780&module=M")

igpm.columns = ["IGP"]
credito_pib.columns = ["CRED/PIB"]
div_liq.columns = ["DIV_LIQ"]

### Unindo valores em um dataframe

In [32]:
from functools import reduce

dfs = [igpm, credito_pib, div_liq, pib, target] 
df = reduce(lambda left, right: left.join(right, how = "inner"), dfs)